In [17]:
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers

### Load the data 

In [5]:
imbd,info= tfds.load('imdb_reviews',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ABC\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteAHUFUG\imdb_reviews-train.t…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ABC\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteAHUFUG\imdb_reviews-test.tf…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\ABC\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteAHUFUG\imdb_reviews-unsuper…

Dataset imdb_reviews downloaded and prepared to C:\Users\ABC\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [6]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset. This is a dataset for binary sentiment
    classification containing substantially more data than previous benchmark
    datasets. We provide a set of 25,000 highly polar movie reviews for training,
    and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='C:\\Users\\ABC\\tensorflow_datasets\\imdb_reviews\\plain_text\\1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
        'text': Text(shape=(), dtype=string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_s

In [7]:
for t in imbd['train'].take(3):
    print(t)

(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

### Data Splitting

In [9]:
train_data,test_data=imbd['train'],imbd['test']
train_sentence=[]
train_label=[]

test_sentence=[]
test_label=[]

for s,l in train_data:
    train_sentence.append(s.numpy().decode('utf8'))
    train_label.append(l.numpy())
    
for s,l in test_data:
    test_sentence.append(s.numpy().decode('utf8'))
    test_label.append(l.numpy())

In [11]:
for l in train_label[:4]:
    print(f"labels of first 4 is {l}")

labels of first 4 is 0
labels of first 4 is 0
labels of first 4 is 0
labels of first 4 is 1


## Genrate Padded Sequences

In [42]:
vocab_Size=10000
max_len=120
embedding_dim=16
trunc='post'
oov_tok='<OOV>'

tokenizer=Tokenizer(num_words=vocab_Size,oov_token=oov_tok)

tokenizer.fit_on_texts(train_sentence)
train_sequence=tokenizer.texts_to_sequences(train_sentence)
test_sequence=tokenizer.texts_to_sequences(test_sentence)

train_sequence=pad_sequences(train_sequence,maxlen=max_len,truncating=trunc)
test_sequence=pad_sequences(test_sequence,maxlen=max_len,truncating=trunc)

In [46]:
training_labels_final = np.array(train_label)
testing_labels_final = np.array(test_label)

In [16]:
print(train_sequence[:3])

[[   0    0    0   12   14   33  425  392   18   90   28    1    9   32
  1366 3585   40  486    1  197   24   85  154   19   12  213  329   28
    66  247  215    9  477   58   66   85  114   98   22 5675   12 1322
   643  767   12   18    7   33  400 8170  176 2455  416    2   89 1231
   137   69  146   52    2    1 7577   69  229   66 2933   16    1 2904
     1    1 1479 4940    3   39 3900  117 1584   17 3585   14  162   19
     4 1231  917 7917    9    4   18   13   14 4139    5   99  145 1214
    11  242  683   13   48   24  100   38   12 7181 5515   38 1366    1
    50  401   11   98 1197  867  141   10]
 [   0    0    0    0    0    0    0    0   11   26   75  571    6  805
  2354  313  106   19   12    7  629  686    6    4 2219    5  181  584
    64 1454  110 2263    3 3951   21    2    1    3  258   41 4677    4
   174  188   21   12 4078   11 1578 2354   86    2   20   14 1907    2
   112  940   14 1811 1340  548    3  355  181  466    6  591   19   17
    55 1817    5   49

## Buidl A model and Complie 

### Functional API

In [39]:
inp=layers.Input(max_len)
x=layers.Embedding(vocab_Size,embedding_dim)(inp)
x=layers.Flatten()(x)
x=layers.Dense(6,activation='relu')(x)
x=layers.Dense(1,activation='sigmoid')(x)

model=tf.keras.Model(inp,x)

In [40]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 120)]             0         
                                                                 
 embedding_11 (Embedding)    (None, 120, 16)           160000    
                                                                 
 flatten_10 (Flatten)        (None, 1920)              0         
                                                                 
 dense_16 (Dense)            (None, 6)                 11526     
                                                                 
 dense_17 (Dense)            (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


### Sequential API

In [41]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_Size, embedding_dim, input_length=max_len),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Setup the training parameters
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 120, 16)           160000    
                                                                 
 flatten_11 (Flatten)        (None, 1920)              0         
                                                                 
 dense_18 (Dense)            (None, 6)                 11526     
                                                                 
 dense_19 (Dense)            (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [49]:
num_epochs = 10

# Train the model
model.fit(train_sequence, training_labels_final, epochs=num_epochs, validation_data=(test_sequence, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.4862 - accuracy: 0.7520 - val_loss: 0.3966 - val_accuracy: 0.8176
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2355 - accuracy: 0.9108 - val_loss: 0.4116 - val_accuracy: 0.8210
Epoch 3/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0906 - accuracy: 0.9770 - val_loss: 0.5078 - val_accuracy: 0.8092
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0219 - accuracy: 0.9970 - val_loss: 0.6181 - val_accuracy: 0.8029
Epoch 5/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0050 - accuracy: 0.9996 - val_loss: 0.6859 - val_accuracy: 0.8089
Epoch 6/10
782/782 [==============================] - 1s 2ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.7444 - val_accuracy: 0.8077
Epoch 7/10
782/782 [==============================] - 1s 2ms/step - loss: 8.0817e-04 - accuracy: 1.0000 - val_loss: 0.7958 - val_accuracy: 0.8084
Ep

### Visualize Word Embeddings

In [57]:
embedding_layer=model.layers[0]
embedding_weights=embedding_layer.get_weights()[0]
print(embedding_weights.shape)

(10000, 16)


You will need to generate two files:

* vecs.tsv - contains the vector weights of each word in the vocabulary
* meta.tsv - contains the words in the vocabulary
* For this, it is useful to have reverse_word_index dictionary so you can quickly lookup a word based on a given index. For * example, reverse_word_index[1] will return your OOV token because it is always at index = 1. Fortunately, the Tokenizer class alre

In [60]:
import io

# Open writeable files
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
reverse_word_index = tokenizer.index_word
# Initialize the loop. Start counting at `1` because `0` is just for the padding
for word_num in range(1, vocab_Size):

  # Get the word associated at the current index
  word_name = reverse_word_index[word_num]

  # Get the embedding weights associated with the current index
  word_embedding = embedding_weights[word_num]

  # Write the word name
  out_m.write(word_name + "\n")

  # Write the word embedding
  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

In [61]:
# Close the files
out_v.close()
out_m.close()

### 'https://projector.tensorflow.org/'